### **Librerías**


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import math
from scipy import stats

from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import PowerTransformer,StandardScaler
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import linear_model
from sklearn.neural_network import MLPRegressor

### **ETL**

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
original_data = pd.read_csv("/content/drive/MyDrive/DNI_Data.csv")
original_data

,Month,DNI,Cloud Type,Dew Point,Solar Zenith Angle,Wind Speed,Relative Humidity,Temperature,Pressure
0,1,0,1,-12,156.44,5.4,67.86,-8,920
1,1,0,4,-12,155.80,5.3,67.84,-8,920
2,1,0,4,-12,153.74,5.3,68.31,-8,920
3,1,0,4,-12,150.58,5.1,68.30,-8,920
4,1,0,4,-12,146.64,5.0,69.20,-8,920
...,...,...,...,...,...,...,...,...,...
17515,12,0,1,-12,141.43,5.2,68.51,-8,920
17516,12,0,1,-12,145.95,5.2,68.30,-8,920
17517,12,0,7,-12,149.99,5.2,68.28,-8,920
17518,12,0,7,-12,153.30,5.2,67.91,-8,920


### **Data Clean and Validation**

In [5]:
# Data Types are Correct, but is better to have strings in Month and Cloud Type
original_data.dtypes 

Month                   int64
DNI                     int64
Cloud Type              int64
Dew Point               int64
Solar Zenith Angle    float64
Wind Speed            float64
Relative Humidity     float64
Temperature             int64
Pressure                int64
dtype: object

In [6]:
original_data['Month']= original_data['Month'].astype('string')
original_data['Cloud Type']= original_data['Cloud Type'].astype('string')

In [7]:
#There are not Nan values
original_data.isnull().sum()

Month                 0
DNI                   0
Cloud Type            0
Dew Point             0
Solar Zenith Angle    0
Wind Speed            0
Relative Humidity     0
Temperature           0
Pressure              0
dtype: int64

### **EDA**

In [ ]:
numeric_columns = original_data.select_dtypes(include=np.number).columns.tolist()
label_columns = ["Month","Cloud Type"]

In [ ]:
original_data.describe()

In [ ]:
original_data.describe(exclude=[np.number])

In [ ]:
k = int(np.ceil(1 + np.log2(original_data.shape[1]))) # Using Sturges Rule to determine the number of bins
original_data[numeric_columns].hist(bins = k, figsize=(12,12),color="purple")
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
original_data.boxplot(vert=False)
plt.show()

In [ ]:
plotf = original_data[numeric_columns + ["Month"]]
sns.pairplot(plotf,hue="Month")
plt.show()

In [ ]:
plotf = original_data[numeric_columns + ["Cloud Type"]]
sns.pairplot(plotf,hue="Cloud Type")
plt.show()

In [ ]:
correlation = original_data[numeric_columns].corr()
plt.figure(figsize = (8,8))
ax = sns.heatmap(correlation,vmax=1, square = True, cmap="inferno",annot=True)

### **Choosing Best Predictors**

In [ ]:
kb = SelectKBest(k="all", score_func=f_regression)
target = "DNI"
xp = original_data[[x for x in original_data[numeric_columns].columns if x!= target]]
yp = original_data[target]

fit = kb.fit(xp, yp.values.ravel())
index = np.argsort(fit.scores_)[::-1]
features = [] # These list are using to store the scores and features
scores = []
for i in range(len(index)):
    features.append(xp.columns[index[i]])
    scores.append(fit.scores_[[index[i]][0]]) #In this part I am store the scores and features

# This dataframe is used to create a better Plot
Best= {'Column': features, 'Scores': scores }  
df = pd.DataFrame(Best)  
Best10= {'Column': features[0:10], 'Scores': scores[0:10] }  
df10 = pd.DataFrame(Best10)  
# Here we plot the Data Frame
sns.barplot(x='Scores',y='Column',data=df10,orient="h",palette="inferno")
plt.show()

### **Check Normality**

In [ ]:
Solar_angle = original_data["Solar Zenith Angle"]
fig, ax = plt.subplots(figsize=(7,4))
sm.qqplot(
    Solar_angle,
    fit   = True,
    line  = 'q',
    alpha = 0.4,
    lw    = 2,
    ax    = ax
)
ax.set_title('QQ Graph of Solar Zenit Angle', fontsize = 10,
             fontweight = "bold")
ax.tick_params(labelsize = 7)

In [ ]:
humidity = original_data["Relative Humidity"]
fig, ax = plt.subplots(figsize=(7,4))
sm.qqplot(
    humidity,
    fit   = True,
    line  = 'q',
    alpha = 0.4,
    lw    = 2,
    ax    = ax
)
ax.set_title('QQ Graph of Relative Humidity', fontsize = 10,
             fontweight = "bold")
ax.tick_params(labelsize = 7)

In [ ]:
temperature = original_data["Temperature"]
fig, ax = plt.subplots(figsize=(7,4))
sm.qqplot(
    temperature,
    fit   = True,
    line  = 'q',
    alpha = 0.4,
    lw    = 2,
    ax    = ax
)
ax.set_title('QQ Graph of Temperature', fontsize = 10,
             fontweight = "bold")
ax.tick_params(labelsize = 7)

### **Split Data**

In [ ]:
xs = original_data[["Solar Zenith Angle","Relative Humidity","Temperature"]]
y = original_data["DNI"]

X_train, X_test, y_train, y_test = train_test_split(xs, y,test_size=0.2,random_state=3)

In [ ]:
print(f"The X train values are {X_train.shape[0]}")
print(f"The X test values are {X_test.shape[0]}")
print(f"The Y values are {y_train.shape[0]}")
print(f"The Y test values are {y_test.shape[0]}")
xs_labels = ["Solar Zenith Angle","Relative Humidity","Temperature"]

### **Preprocesing Data**

In [ ]:
pipeline = Pipeline([
      ("Scaler", StandardScaler()),
      ("Transformation", PowerTransformer(method='yeo-johnson'))
])

preprocessing_pipe = ColumnTransformer([
      ("pipeline",pipeline,xs_labels),
])

In [ ]:
X_train = preprocessing_pipe.fit_transform(X_train)
X_test  = preprocessing_pipe.transform(X_test)

### **Models and Validation**

- **Linear Regression**

In [ ]:
LR = linear_model.LinearRegression()
LR.fit(X_train,y_train.values.ravel())

In [ ]:
y_pred = LR.predict(X_test)

print('Coefficients:', LR.coef_)
print('Intercept:', LR.intercept_)
print('Mean squared error (MSE): %.2f'
      % mean_squared_error(y_test, y_pred))
print('Root Mean squared error (RMSE): %.2f'
      % math.sqrt(mean_squared_error(y_test, y_pred)))
print('Coefficient of determination (R^2): %.2f'
      % r2_score(y_test, y_pred))


- **Ridge Regression**

In [ ]:
LRR = linear_model.Ridge(0.3) #0.1, 0.3, 0.5, 1
LRR.fit(X_train,y_train.values.ravel())

In [ ]:
y_pred_ridge = LRR.predict(X_test)

print('Coefficients:', LRR.coef_)
print('Intercept:', LRR.intercept_)
print('Mean squared error (MSE): %.2f'
      % mean_squared_error(y_test, y_pred_ridge))
print('Root Mean squared error (RMSE): %.2f'
      % math.sqrt(mean_squared_error(y_test, y_pred)))
print('Coefficient of determination (R^2): %.2f'
      % r2_score(y_test, y_pred_ridge))

- **Neural Network**

In [ ]:
NN = MLPRegressor(random_state=1, max_iter=2000)
NN.fit(X_train, y_train.values.ravel())

In [ ]:
y_pred_NN = NN.predict(X_test)
a = mean_squared_error(y_test, y_pred_NN)
b = math.sqrt(a)
r = r2_score(y_test, y_pred_NN)

print('Mean squared error (MSE): %.2f'
      % a)
print('Root Mean squared error (RMSE): %.2f'
      % b)
print('Coefficient of determination (R^2): %.2f'
      % r)
